## Decorators


In this lesson, we are going to take a look at Decorators in Python. We will review them on the surface. The best way to learn them is that we introduce them, and then you go through the practicals and try to find as many challenges on this topic as possible. For example [here](https://github.com/IvanYingX/Challenges_AiCore.git)

This lesson will use a folder name `utils`. If you are in Colab and don't have that folder right now, run the following code to download the folder with the examples. Remember that you can access `.py` files in Colab and modify them!

In [ ]:
!wget "https://aicore-files.s3.amazonaws.com/Foundations/Python_Programming/advanced_py.zip"
import zipfile
with zipfile.ZipFile("advanced_py.zip", 'r') as zip_ref:
    zip_ref.extractall("utils")

To understand decorators, you should understand that inner functions also exist

In [1]:
def caller(num):

    def first_child():
        print('I am being called')
        return "I am the first child"

    def second_child():
        return "I am the second child"

    if num == 1:
        return first_child
    else:
        return second_child


print(caller(1))

<function caller.<locals>.first_child at 0x00000245E006ACB0>


Observe the output above. It is returning a function rather than a string or a number. Try to follow the flow of the code and see what happens.

1. We call for caller giving the argument a value of 1
2. Then, INSIDE the function, we define two functions: first_child and second_child. But we don't call them yet! so nothing within them is executed.
3. We keep executing the caller function, and trigger the if statement that checks if the argument is 1.
4. We return the function, but JUST the function, not the result of the function.

With that in mind, let's see decorators. 

> <font size=+1>Decorators extend the functionality of a function.</font>

Let's see an example. First we define a function with a nested function. The name of the outer function is something we can decide to suit whatever we are going to use the decorator for. But, as a convention, the inner function is called `wrapper`.

In this case, we have a function that takes as an argument another function. The inner function is defined but not called! and after that, the inner function is returned.

In [2]:
def my_decorator(func):
    def wrapper():
        print("I come before the function!")
        func()
        print("I come after the function!")
    return wrapper

def say_truth():
    print(f'Epstein didn\'t kill himself')

So, if we call for the outer function, we get the inner function (but not its result!).

In [3]:
my_decorator(say_truth)

<function __main__.my_decorator.<locals>.wrapper()>

Observe the output: `<function __main__.my_decorator.<locals>.wrapper()>`. This means that the output of the outer function is the inner function (I know, it sounds weird, but it is correct).

Let's assign that function to a variable, and then call it

In [4]:
my_func = my_decorator(say_truth)
my_func()

I come before the function!
Epstein didn't kill himself
I come after the function!


Now, the output of the inner function is printed! Basically, `my_func` contained the information about `wrapper` and `say_truth`, so once called, we could see how `wrapper` interacted with `say_truth`.

Let's use the same decorator with another function.

In [5]:
def wave():
    print('Hello, world!')

In [6]:
my_wave_func = my_decorator(wave)
my_wave_func()

I come before the function!
Hello, world!
I come after the function!


Observe that `my_wave_func` contains information about `wrapper` and `wave`, meaning that we are able to see how `wrapper` interacts with `wave`.

### Decorators with Arguments

So far, we have seen decorators that take no arguments. But what if we want to pass arguments to the decorator? Let's see what happens if to the current decorator, we pass a function with an argument.

In [7]:
def say_hello(name):
    print(f'Hello, {name}!')

In [8]:
my_hello_func = my_decorator(say_hello)

Cool! So far so good, no errors, let's try calling it.

In [9]:
my_hello_func()

I come before the function!


TypeError: say_hello() missing 1 required positional argument: 'name'

Can you see what happened? The output prints `I come before the function` but then it throws an error. Remember that `func()` is the function we passed to the decorator, in this case, `say_hello`, which in turn, expects an argument. In other words, it's like we had this inside `my_decorator`:

``` python
    def wrapper():
        print("I come before the function!")
        say_hello()
        print("I come after the function!")
```

But `say_hello` needs an argument, so we get an error.

Thus, what we can do is to write the decorator to expect an argument from the caller

In [10]:
def my_decorator(func):
    def wrapper():
        print("I come before the function!")
        func(name)
        print("I come after the function!")
    return wrapper

my_hello_func = my_decorator(say_hello)

Hmmm, something looks wrong. Let's see what happens if we try to run it

In [11]:
my_hello_func()

I come before the function!


NameError: name 'name' is not defined

Right! The function is in the local scope, and it doesn't know any variable called `name`. Thus, we need to pass `name` to the wrapper

In [12]:
def my_decorator(func):
    def wrapper(name):
        print("I come before the function!")
        func(name)
        print("I come after the function!")
    return wrapper

my_hello_func = my_decorator(say_hello)

In [13]:
my_hello_func('Ivan')

I come before the function!
Hello, Ivan!
I come after the function!


Good! Now it looks like it's working fine. Observe that in the last call, we pass `Ivan` as an argument, because remember that `my_hello_func` is the `wrapper` with information of `say_hello`, and `wrapper` now needs and argument

Ok, so, why is this useful? Well, we can use it to pass arguments to the function. For example, we can pass a name to the function, and then we can use that name to greet the user. Or something that might be more useful, we can measure the time it takes for a function to execute.

In [14]:
import time

def my_timer(func):
    def wrapper():
        time_0 = time.time()
        func()
        time_1 = time.time()
        print(f'It took {time_1 - time_0} second to run')
    return wrapper


def dummy_fun():
    for _ in range(50000000):
        x = 'I am just losing your time'
    return x

time_exec = my_timer(dummy_fun)
print(time_exec)

<function my_timer.<locals>.wrapper at 0x00000245E1C1C5E0>


Good, so we have a wrapper that can measure the time it takes to execute a function. Let's call it!

In [15]:
time_exec()

It took 3.9159905910491943 second to run


But an even cooler thing you can do is this syntactic sugar offered by Python. You can decorate a function by adding `@` when defining the function

In [16]:
@my_timer
def dummy_fun():
    for _ in range(50000000):
        x = 'I am just losing your time'

dummy_fun()

It took 3.310981512069702 second to run


### Multiple Arguments in a Decorator

So we have just seen that you can extend the functionality of a function using decorators. But what if we want to extend it with multiple arguments? We don't know what functions our decorator is going to wrap, so there might be one or many arguments. What can we do?

The key is in these two guys *args **kwargs

In [17]:
def repeat(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper

def say_hi(name):
    print(f'Hello {name}')
    return 1

repeat(say_hi)('Ivan')

Hello Ivan
Hello Ivan


### `Return` statement in the wrapper

Imagine that we want to return something from the decorated function.

Let's define a small function: factorial, that returns the factorial of a number. (The factorial of a number is the product of all the numbers from 1 to that number)

In [18]:
def factorial(n):
    previous = 1
    for i in range(1, n + 1):
        previous *= i 
    return previous

print(factorial(5))

120


Let's define again a timer decorator, but in this case, the decorated function can accept any argument (so we use `*args` and `**kwargs`)

In [19]:
def my_timer(fun):
    def wrapper(*args, **kwargs):
        time_0 = time.time()
        fun(*args, **kwargs)
        time_1 = time.time()
        print(f'It took {time_1 - time_0} second to run')
    return wrapper


@my_timer
def factorial(n):
    previous = 1
    for i in range(1, n + 1):
        previous *= i
    return previous

print(factorial(5))

It took 0.0 second to run
None


We are printing the message about how long it took for it to run, but we are not getting the value of factorial. That is because wrapper is not returning anything, and thus, it prints None. The solution is simply put a `return` statement. Easy, isn't it...? 

In [20]:
def my_timer(fun):
    def wrapper(*args, **kwargs):
        time_0 = time.time()
        output = fun(*args, **kwargs)
        time_1 = time.time()
        print(f'It took {time_1 - time_0} second to run')
        return output
    return wrapper


@my_timer
def factorial(n):
    previous = 1
    for i in range(1, n + 1):
        previous *= i
    return previous

print(factorial(5))

It took 0.0 second to run
120


### Classmethods and Staticmethods

Two often used decorators are classmethods and staticmethods. They add functionality to methods

classmethod must have a reference to a class object instead of an instance, whereas static methods doesn't point to an instance or a class (however they are bound to the class, so you have to call it from that class or from an instance of that class)

Let's observe an example where we use classmethod and staticmethod. We define a class `Date` which we can initialize with three numbers: day, month and year. It also has a method `get_date` that returns the date in a string format.

In [21]:
class Date:

    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year
    
    def get_date(self):
        print(f'The date is {self.day}/{self.month}')

date = Date(19, 5, 1991)
date.get_date()

The date is 19/5


Nothing fancy so far. Let's add a classmethod to `Date`

> <font size=+1>Classmethods are used to create methods that don't depend on the instance but on the class</font>

This means that the classmethod can be called without an instance of the class. Actually, when a method is a classmethod it will be called before the `__init__` method

Ok, that is cool, but why would we want to do that? Well, imagine that you want to create an instance of `Date` that directly tells you the current date. You can do that by calling a classmethod that returns the current date, and you don't have to pass any input.

The syntax of a classmethod is:
```
@classmethod
def method_name(cls, *args, **kwargs):
    # do something
    return something
```

Important! Notice that the first argument of the classmethod is the class itself, not an instance of the class. So we don't use `self`, but instead `cls`.

In [22]:
import datetime

class Date:
    
    def __init__(self, day=0, month=0, year=0):
        print('__init__ method called!')
        self.day = day
        self.month = month
        self.year = year

    def get_date(self):
        print(f'The date is {self.day}/{self.month}')

    @classmethod
    def today(cls):
        print('Classmethod called!')
        today = datetime.date.today()
        day = today.day
        month = today.month
        year = today.year
        print('I am about to call the __init__ method')
        return cls(day, month, year)

Observe the code above, (I just added a few of print statements to make it easier to understand). We are creating a classmethod that returns the current date, first by calculating the current date, and then, the method is calling for the constructor of the class `Date` with the arguments we calculated.

We instansiate a class `Date` using its classmethod like this: (Observe what is printed out and compare the output to follow the flow of the code)

In [23]:
current_date = Date.today()

Classmethod called!
I am about to call the __init__ method
__init__ method called!


In [24]:
current_date.get_date()

The date is 15/5


### Staticmethods

> <font size=+1>Staticmethods are methods that are not bound to an instance or a class.</font>

This means that we DON'T need the `self` or `cls` arguments, because anything we pass to the staticmethod won't depend on the instance or the class.

Same as classmethods, we can define staticmethods like this:
```
@staticmethod
def method_name(*args, **kwargs):
    # do something
    return something
```

Let's define a staticmethod that returns checks if the date is valid. As you can imagine, checking if a date is valid doesn't depend on the instance or the class, so staticmethods are perfect for this.

In [25]:
class Date:

    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year
        
    def get_date(self):
        print(self.day, self.month)

    @classmethod
    def today(cls):
        today = datetime.date.today()
        day = today.day
        month = today.month
        year = today.year
        return cls(day, month, year)
  
    @staticmethod
    def is_date_valid(day, month, year):
        return day <= 31 and month <= 12 and year <= 3999

Once again, staticmethods don't need `self` or `cls` arguments, meaning that we can call them from an instance or from the class.

In [26]:
Date.is_date_valid(1, 1, 1)

True

In [27]:
date = Date.today()
date.is_date_valid(1, 1, 1)

True

One cool thing about this is that we can use the staticmethod inside the `__init__` method of the class `Date`, so when we pass the arguments to the constructor, we can use the staticmethod to check if the date is valid.

In [28]:
class Date:

    def __init__(self, day=0, month=0, year=0):
        if self.is_date_valid(day, month, year):
            self.day = day
            self.month = month
            self.year = year
        else:
            raise ValueError('Invalid date!')
        
    def get_date(self):
        print(self.day, self.month)

    @classmethod
    def today(cls):
        today = datetime.date.today()
        day = today.day
        month = today.month
        year = today.year
        return cls(day, month, year)
  
    @staticmethod
    def is_date_valid(day, month, year):
        return day <= 31 and month <= 12 and year <= 3999

If we pass a valid date, the instance will be created and no problems will appear. On the other hand, if we try to use an invalid date, Python will raise an error (ValueError)

In [29]:
good_date = Date(19, 5, 1991)

No problem so far

In [30]:
bad_date = Date(31, 13, 2021)

ValueError: Invalid date!

Pretty neat isn't it?